In [12]:
import findspark
findspark.init()

import pyspark

from src.emulator import generate_transactions_table, add_frauds, generate_dataset

In [13]:
customer_profiles_table_df, terminal_profiles_table_df, transactions_df = generate_dataset()

Time to generate customer profiles table: 0.095s
Time to generate terminal profiles table: 4.7s
Time to associate terminals to customers: 1.2e+02s
Time to generate transactions: 1.1e+02s


In [3]:
transactions_df = add_frauds(customer_profiles_table_df, terminal_profiles_table_df, transactions_df)

Number of frauds from scenario 1: 1306
Number of frauds from scenario 2: 12755
Number of frauds from scenario 3: 6271


In [4]:
customer_profiles_table_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   CUSTOMER_ID          5000 non-null   int64  
 1   x_customer_id        5000 non-null   float64
 2   y_customer_id        5000 non-null   float64
 3   mean_amount          5000 non-null   float64
 4   std_amount           5000 non-null   float64
 5   mean_nb_tx_per_day   5000 non-null   float64
 6   available_terminals  5000 non-null   object 
 7   nb_terminals         5000 non-null   int64  
dtypes: float64(5), int64(2), object(1)
memory usage: 312.6+ KB


In [5]:
terminal_profiles_table_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   TERMINAL_ID    10000 non-null  int64  
 1   x_terminal_id  10000 non-null  float64
 2   y_terminal_id  10000 non-null  float64
dtypes: float64(2), int64(1)
memory usage: 234.5 KB


In [11]:
transactions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2395956 entries, 0 to 2395955
Data columns (total 9 columns):
 #   Column             Dtype         
---  ------             -----         
 0   TRANSACTION_ID     int64         
 1   TX_DATETIME        datetime64[ns]
 2   CUSTOMER_ID        object        
 3   TERMINAL_ID        object        
 4   TX_AMOUNT          float64       
 5   TX_TIME_SECONDS    object        
 6   TX_TIME_DAYS       object        
 7   TX_FRAUD           int64         
 8   TX_FRAUD_SCENARIO  int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(4)
memory usage: 164.5+ MB


In [7]:
spark = (
    pyspark.sql.SparkSession.builder
        .appName("notebook-create")
        .master("yarn")
        .config("spark.executor.memory", "1g")
        .config("spark.driver.memory", "1g")
        .getOrCreate()
)
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

In [8]:
transactions_sdf = spark.createDataFrame(transactions_df) 
transactions_sdf.printSchema()

terminal_profiles_table_sdf = spark.createDataFrame(terminal_profiles_table_df) 
terminal_profiles_table_sdf.printSchema()

customer_profiles_table_sdf = spark.createDataFrame(customer_profiles_table_df) 
customer_profiles_table_sdf.printSchema()

/usr/lib/spark/python/pyspark/sql/pandas/conversion.py:327: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


root
 |-- TRANSACTION_ID: long (nullable = true)
 |-- TX_DATETIME: timestamp (nullable = true)
 |-- CUSTOMER_ID: long (nullable = true)
 |-- TERMINAL_ID: long (nullable = true)
 |-- TX_AMOUNT: double (nullable = true)
 |-- TX_TIME_SECONDS: long (nullable = true)
 |-- TX_TIME_DAYS: long (nullable = true)
 |-- TX_FRAUD: long (nullable = true)
 |-- TX_FRAUD_SCENARIO: long (nullable = true)

root
 |-- TERMINAL_ID: long (nullable = true)
 |-- x_terminal_id: double (nullable = true)
 |-- y_terminal_id: double (nullable = true)

root
 |-- CUSTOMER_ID: long (nullable = true)
 |-- x_customer_id: double (nullable = true)
 |-- y_customer_id: double (nullable = true)
 |-- mean_amount: double (nullable = true)
 |-- std_amount: double (nullable = true)
 |-- mean_nb_tx_per_day: double (nullable = true)
 |-- available_terminals: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- nb_terminals: long (nullable = true)



In [9]:
transactions_sdf.write.mode("append").parquet("hdfs:///user/otus/card-fraud-detection/transactions.parquet")
terminal_profiles_table_sdf.write.mode("append").parquet("hdfs:///user/otus/card-fraud-detection/terminal_profiles_table.parquet")    
customer_profiles_table_sdf.write.mode("append").parquet("hdfs:///user/otus/card-fraud-detection/customer_profiles_table.parquet")

In [10]:
spark.stop()